In [1]:
import pandas as pd
import numpy as np

from google.cloud import bigquery
# import pandas_gbq

client = bigquery.Client()

In [2]:
start_date = '2024-04-09'
end_date =  '2024-04-15'

# Create a date range
date_range = pd.date_range(start=start_date, end=end_date)


In [3]:
# MAU_query =  ''
# for date in date_range:
#     dd = date.date()
#     MAU_query += f"""SELECT '{dd}' as Date,
#   count ( distinct customer_id) as MAU
# FROM
#   `prj-prod-dataplatform.risk_mart.customer_transactions` 
# WHERE

#   transaction_date BETWEEN DATE_SUB('{dd}', INTERVAL 30 DAY) AND '{dd}'
#   AND ABS(trx_amount) > 100
#   AND account_type = "Tonik Account" 

#   UNION ALL
#   """
    
# MAU_query = MAU_query[:-15]

In [11]:
# MAU_query =  ''
# for date in date_range:
#     dd = date.date()
#     MAU_query += f"""SELECT 
#     '{dd}' as Date,
#     COUNT(DISTINCT customer_id) as MAU
# FROM 
#     `prj-prod-dataplatform.risk_mart.customer_transactions` 
# WHERE
#     transaction_date BETWEEN DATE_SUB('{dd}', INTERVAL 30 DAY) AND '{dd}'
#     AND ABS(trx_amount) > 100
#     AND account_type = "Tonik Account" 
#     AND customer_id NOT IN (
#         SELECT 
#             CAST(customerId AS STRING)
#         FROM 
#             `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
#         WHERE 
#             loanPaidStatus IN ('Normal', 'In Arrears'))

#   UNION ALL
#   """
    
# MAU_query = MAU_query[:-15]

In [12]:
MAU = client.query(MAU_query).to_dataframe()


In [13]:
# MAU['date'] = pd.to_datetime(MAU['date'])

In [14]:
MAU = MAU.sort_values('Date')

In [15]:
MAU['Change in MAU'] = MAU['MAU'].pct_change()

In [16]:
MAU

,Date,MAU,Change in MAU
0,2024-04-09,31230,<NA>
4,2024-04-10,31238,0.000256
6,2024-04-11,31315,0.002465
1,2024-04-12,31340,0.000798
5,2024-04-13,31324,-0.000511
3,2024-04-14,31178,-0.004661
2,2024-04-15,31061,-0.003753


In [17]:
MAU.to_excel('MAU_non_Normal_In_Arrears.xlsx',index=False)

In [ ]:
WITH 
    TSA_customers --list of customers having TSA account
        AS (
        SELECT DISTINCT OFCUSTOMERID, OFDATEOPENED, OFDATECLOSED 
        FROM prj-prod-dataplatform.core_raw.customer_accounts 
        WHERE OFPRODUCTCATEGORY = 'savings' 
        AND CRINTERDESC = 'Transactional Savings Account Inv_R'
    ),
    loan_cust --list of customers having active loan
        AS (
        SELECT 
            CAST(customerId AS STRING) customerId
        FROM 
            `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
        WHERE 
            loanPaidStatus IN ('Normal', 'In Arrears')
    ),
    MAU --list of monthly active users
        AS (
        SELECT DISTINCT customer_id 
        FROM `prj-prod-dataplatform.risk_mart.customer_transactions`
        WHERE transaction_date BETWEEN DATE_SUB('2024-04-13', INTERVAL 30 DAY) AND '2024-04-13'
        AND ABS(trx_amount) > 100
        AND account_type = "Tonik Account"
        AND customer_id NOT IN (SELECT customerId FROM loan_cust)
    ),
    MAU_count --count of monthly active users
        AS (
        SELECT
            '2024-04-13' AS date,
            COUNT(DISTINCT customer_id) AS MAU
        FROM MAU
    ),
    TSA_non_loan_cust --count of TSA customers whose account is not closed and no active loan
        AS (
        SELECT 
            '2024-04-13' AS date,
            COUNT(DISTINCT OFCUSTOMERID) AS TSA_customers 
        FROM 
            TSA_customers 
        WHERE 
            DATE('2024-04-13') BETWEEN DATE(OFDATEOPENED) AND 
            (CASE WHEN DATE(OFDATECLOSED) = '1970-01-01' THEN DATE('2024-04-13') ELSE DATE(OFDATECLOSED) END)  
        AND OFCUSTOMERID NOT IN (SELECT customerId FROM loan_cust)
    ),
    TSA_balance --total TSA balance on particular date
        AS (
        SELECT 
            CAST(balanceDateAsOf AS STRING) AS date,
            SUM(clearedbalance) AS TSA_balance 
        FROM `prj-prod-dataplatform.risk_mart.customer_balance` 
        WHERE 
            balanceDateAsOf = "2024-04-13" 
            AND account_type  = "Tonik Account"
            AND client_id NOT IN (SELECT customerId FROM loan_cust)
        GROUP BY 1
    ),
    TSA_balance_MAU --total TSA balance for monthly active users on particular date
        AS (
        SELECT 
            CAST(balanceDateAsOf AS STRING) AS date,
            SUM(clearedbalance) AS TSA_balance_MAU 
        FROM `prj-prod-dataplatform.risk_mart.customer_balance` 
        WHERE 
            balanceDateAsOf = "2024-04-13" 
            AND account_type  = "Tonik Account"
            AND client_id IN (SELECT customer_id FROM MAU)
        GROUP BY 1
    ),
    transactional_cust --Mareks criteria
        AS (
        SELECT 
            DISTINCT customer_id
        FROM 
            `prj-prod-dataplatform.risk_mart.customer_transactions`
        WHERE
            transaction_date BETWEEN DATE_SUB('2024-04-13', INTERVAL 3 MONTH) AND '2024-04-13'
            AND customer_id NOT IN (SELECT customerId FROM loan_cust) 
        GROUP BY
            customer_id
        HAVING 
            COUNT(*) >= 5
    ),
    cust_with_balance --Mareks criteria
        AS (
        SELECT 
            DISTINCT client_id 
        FROM 
            `prj-prod-dataplatform.risk_mart.customer_balance`
        WHERE
            clearedbalance > 1000
            AND account_type IN ("Tonik Account","Group Stash","Individual Stash")
            AND balanceDateAsOf = '2024-04-13'
            AND client_id NOT IN (SELECT customerId FROM loan_cust)
    ),
    cust_with_transactions_and_balance --Mareks criteria
        AS (
        SELECT 
            customer_id 
        FROM 
            transactional_cust
        UNION DISTINCT
        SELECT 
            client_id AS customer_id
        FROM 
            cust_with_balance
    ),
    count_cust_with_transactions_and_balance 
        AS (
        SELECT 
            '2024-04-13' AS date,
            COUNT(customer_id) AS MMAU 
        FROM 
            cust_with_transactions_and_balance
    ),
   b 
    AS (
        SELECT 
            a.date,
            MAU,
            TSA_customers,
            TSA_balance,
            TSA_balance_MAU,
            MMAU
        FROM 
            MAU_count a 
        LEFT JOIN 
            TSA_non_loan_cust b ON a.date = b.date
        LEFT JOIN 
            TSA_balance c ON a.date = c.date
        LEFT JOIN 
            TSA_balance_MAU d ON a.date = d.date
        LEFT JOIN 
            count_cust_with_transactions_and_balance e ON a.date = e.date
    )

SELECT  
    *, 
    MAU/TSA_customers AS Share_of_active_TSA_customers,
    TSA_balance/TSA_customers AS Average_TSA_balance_per_customer,
    TSA_balance_MAU/MAU AS Average_TSA_balance_per_ACTIVE_customer 
FROM 
    b;
